# Notebook to Move final dcm files selected for processing to Project Folder

In [1]:
import shutil
import pickle
import pandas as pd
from l3finder.ingest import *
import os
import multiprocessing
from multiprocessing import get_context
from multiprocessing import set_start_method
from tqdm.notebook import tqdm
from investigate import *

# Custom functions
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):        
    with open(filename, 'rb') as input:
        return pickle.load(input)

In [2]:
cwd = os.getcwd()
output = '/tf/pickles'
data = '/tf/data'
target = '/tf/output'
dump = 1

In [3]:
df_final = load_object(os.path.join(output,'df_final_dump_1.pkl'))
subjects = load_object(os.path.join(output,'subjects_final_dump_1.pkl'))

In [4]:
file_mover = functools.partial(
        move_subject,
        df_final=df_final,
        target = target)

In [5]:
df_final = df_final[df_final['ID'].isin([s.id_ for s in subjects])]
print('Len of df: ', len(df_final))
print('Len of sub: ', len(subjects))

Len of df:  2052
Len of sub:  2052


In [6]:
if __name__=='__main__':
    # Find series images
    print("Moving valid subjects")
    
    def pool_mover(pool, func, candidates):
        return [a for a in tqdm(pool.imap_unordered(func, candidates),total=len(candidates))]
        
    print('Filtering series using ', 10, ' cores:')
        
    with get_context("spawn").Pool(processes=10) as p:
        result_list = pool_mover(p, file_mover,subjects)
        p.close()
        p.join()
        
    df_final['Processed'] = result_list

Moving valid subjects
Filtering series using  10  cores:


In [7]:
display(df_final.head(10))

,ID,Axial,Sagittal,Overlap,MissingScore,PairValidity,AxSlices,SagSlices,QC,Processed
2,Z1001265,Z1001265-SE-4-Body_Std._Axial_AIDR_-_H_5.0,Z1001265-SE-7-Body_Std._Axial_AIDR_-_H_Sagitta...,0.998,1.011,2.009,90,99,hq,1
3,Z1001413,Z1001413-SE-5-Axial_Body_5.0,Z1001413-SE-8-Sagittal_Body_Sagittal_3.000,0.989,1.014,2.003,70,79,hq,1
4,Z1001998,Z1001998-SE-5-Axial_Body_5.0,Z1001998-SE-8-Sagittal_Body_Sagittal_3.000,0.994,1.014,2.008,68,76,hq,1
7,Z1003508,Z1003508-SE-2-Stnd_Pediatric_5.0_CE,Z1003508-SE-7-Stnd_Pediatric_3.0_CE,0.994,1.013,2.007,73,86,hq,1
8,Z1003895,Z1003895-SE-2-Axial_Std_5.0_CE,Z1003895-SE-7-Sagittal_Sagittal_3.0_CE,0.99,1.021,2.011,58,33,hq,1
9,Z1004285,Z1004285-SE-5-Axial_Body_5.0,Z1004285-SE-8-Sagittal_Body_Sagittal_3.000,0.986,1.014,2,70,80,hq,1
12,Z1008427,Z1008427-SE-7-Std_Pediatric_5.0_CE,Z1008427-SE-10-Std_Pediatric_3.0_CE,0.997,1.014,2.011,75,67,hq,1
13,Z1008758,Z1008758-SE-4-Body_Std._Axial_AIDR_-_H_5.0,Z1008758-SE-7-Body_Std._Axial_AIDR_-_H_Sagitta...,0.994,1.013,2.007,70,92,hq,1
17,Z1009724,Z1009724-SE-4-Axial_Std_3.0,Z1009724-SE-16-Sagittal_Sagittal_3.0_CE,0.528,1.011,1.539,128,54,hq,1
19,Z1011428,Z1011428-SE-2-Body_Std._Axial_AIDR_-_H_5.0,Z1011428-SE-5-Body_Std._Axial_AIDR_-_H_Sagitta...,0.99,1.012,2.002,80,93,hq,1


In [8]:
save_object(df_final,os.path.join(output,'df_final_dump1_processed.pkl'))